# Imports


In [61]:
import minari
import torch

In [60]:
dataset = minari.load_dataset("D4RL/pointmaze/large-v2")

# Prior Functionality

### Getting functions/methods from other classes

In [3]:
class TrajectorySet: 
    def __init__(self): 
        """
        trajectories: a dictionary housing all of the trajectories. The dictionary structure is: 
            {
                1: [trajectory, length of trajectory]
                2: [ ... ]
                etc...
            } 

        num_trajectories: the number of trajectories currently in the set. 
        """

        self.trajectories = {} 
        self.num_trajectories = 0 
    
    def add_trajectory(self, trajectory):
        self.trajectories[self.num_trajectories] = [trajectory, len(trajectory)] 
        self.num_trajectories += 1
    
    def get_num_trajectories(self):
        return self.num_trajectories

    def get_trajectory(self, index): 
        assert index < self.num_trajectories, "Specified index is too large."
        return self.trajectories[index]
    
    def get_trajectory_set(self): 
        return self.trajectories
    
    def get_total_states(self): 
        sum = 0
        for _, v in self.trajectories.items(): 
            sum += v[1]
        return sum 

    def generate_trajectories(self, n_trajectories: int = 2): 
        """
        Generates a specified number of trajectories and saves them into the TrajectorySet class. 

        This runs the scripted agent, where the agent uses a PD controller to follow a 
        path of waypoints generated with QIteration until it reaches the goal.
        """
        ep_data = dataset.sample_episodes(n_episodes=n_trajectories) # sample trajectories

        # adds all of the sampled trajectories into the TrajectorySet 
        for i in range(len(ep_data)):
            ep = ep_data[i] 

            # Note: only saving states since we only need state representations in the encoder 
            self.add_trajectory(ep.observations["observation"]) 

# Sampler

### Need to sample and get positive and negative pairs (states). 

In [62]:
class Sampler(): 
    def __init__(self, T: TrajectorySet): 
        self.T = T 

    def sample_anchor_state(self, t: list) -> tuple[list, int]: 
        """
        Given a trajectory, we sample the anchor state s_i uniformly. 

        Args: 
            t: The given trajectory we sample from. 

        Returns: 
            A tuple containing [s_i, idx]
            s_i: The state that is sampled, represented as a list of (x,y) coordinates and velocities. 
            idx: The time step of s_i. 
        """
        idx = torch.randint(low=0, high=len(t), size=(1,)).item()
        s_i = t[idx] 
        return [s_i, idx]

    def sample_positive_pair_gaussian(self, t: list, anchor_state: tuple[list, int]) -> tuple[list, int]: 
        """
        Given the same trajectory that s_i was sampled from, 
        center a gaussian distribution around s_i to get obtain its positve pair: s_j. 
        
        Reference: https://docs.pytorch.org/docs/stable/generated/torch.normal.html

        Args: 
            t: The given trajectory, which must be the same as the trajectory that was used to sample the anchor state. 
            anchor_state: The anchor state; a tuple containing [s_i, idx].
            s_i: The state itself.
            idx: The time step of s_i.
        
        Return: 
            A tuple containing [s_j, idx]
            s_j: The state that is sampled, represented as a list of (x,y) coordinates and velocities. 
            idx: The time step of s_j.    
        """
        std = 15  # we use 15 to replicate the paper's hyperparams

        _, si_idx = anchor_state

        while True: 
            sj_idx = torch.normal(mean=si_idx, std=std, size=(1,))
            sj_idx = int(sj_idx) 

            # Ensures we don't choose an index out of range or the same state. 
            if (sj_idx < len(t)) and (sj_idx != si_idx): 
                break 
        
        s_j = t[sj_idx] 

        return [s_j, sj_idx]
    
    def sample_batch(self, batch_size=1024, k=2) -> list[tuple]: 
        """ 
        Creates a batch of anchor states, their positive pairs, and negative pairs. 
        There will be 2(batch_size - 1) amount of negative examples per positive pair.

        Args: 
            T: The trajectory set class (must be empty). 
            batch_size: The size of the batch to be generated.
            k: A hyperparameter that dictates the average number of 
                positive pairs sampled from the same trajectory. The 
                lower the number, the lesser the chance of false negatives. 
        
        Returns: 
            A list of tuples containing the anchor_state and its positive pair. 
            The list is the same length as batch_size. 
        """ 

        batch = [] 

        # Generate trajectory set 
        n_trajectories = batch_size // k
        self.T.generate_trajectories(n_trajectories= n_trajectories)

        for _ in range(batch_size): 
            # Sample anchor state 
            rng = torch.randint(low=0, high=n_trajectories, size=(1,)).item() 
            t = self.T.get_trajectory(index=rng)[0]
            
            anchor_state = self.sample_anchor_state(t) 

            # Sample positive pair 
            positive_pair = self.sample_positive_pair_gaussian(t, anchor_state=anchor_state)

            # Retrieve states; time-steps aren't necessary. 
            s_i = anchor_state[0]
            s_j = positive_pair[0]

            batch.append([s_i, s_j]) 

        return batch 

In [64]:
"""
TESTING CELL!
"""
T = TrajectorySet() 
s = Sampler(T)

T.generate_trajectories(n_trajectories=1) 
t = T.get_trajectory(0)[0] 
anchor_state = s.sample_anchor_state(t)
print(f"s_i: {anchor_state[0]}, at time step: {anchor_state[1]}")

positive_pair = s.sample_positive_pair_gaussian(t=t, anchor_state=anchor_state)
print(f"s_j: {positive_pair[0]}, at time step: {positive_pair[1]}")


s_i: [ 1.17242363 -1.09449604  3.9643746   0.55220741], at time step: 270
s_j: [ 0.67423713 -1.1487629   3.12098452 -0.07447397], at time step: 258


In [92]:
"""
TESTING CELL! 

Testing to see if sample_batch works properly. 
"""

batch = s.sample_batch(batch_size=128, k=2)
print("Batch:", batch)
len(batch)

Batch: [[array([ 0.43052296,  0.9267142 , -1.13217762, -1.61046076]), array([ 0.45740177,  0.98231113, -0.42239722, -2.33993079])], [array([ 1.21506283, -1.13491601, -4.88130794, -0.79929498]), array([ 1.95084061, -1.05848305, -5.05672894, -0.59887764])], [array([-1.60220809,  2.22973965,  1.44133977,  4.93941334]), array([-1.6302165 ,  2.13322079,  1.58627366,  4.48499069])], [array([-1.44779685,  0.85795925, -3.07488006,  0.1641953 ]), array([-0.53400015,  0.81762898, -3.26613925, -0.69935363])], [array([ 0.45924773,  1.83041814, -0.46222761, -3.82957198]), array([ 0.37131476,  1.25503114, -0.55339197, -4.55693279])], [array([-3.49436708, -2.24836081, -0.06429648,  3.28498353]), array([-3.4992457 , -2.10473302, -0.37884876,  3.07497176])], [array([ 2.41816396, -2.28475459,  0.12931201, -4.06683792]), array([ 2.41687084, -2.24408621, -0.07129039, -3.83781436])], [array([-1.54632538,  1.00608914,  3.07119032, -0.18803903]), array([-0.80674649,  0.90044196,  4.89868991, -0.52578401])], 

128